In [12]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [13]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })
  
  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset
      
      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()

acc,▁▆▃▇█▆█▇
loss,█▇▃▃▃▂▂▁
acc,0.76317
loss,0.15103


acc,▁▄▁▇██▇▆
loss,█▄▅▅▄▁▃▂
acc,0.80546
loss,0.15339


acc,▁▃▆▆▆█▇█
loss,█▆▂▃▂▂▁▁
acc,0.79556
loss,0.19723


acc,▁▆▆▆▇▇██
loss,█▆▃▃▂▂▂▁
acc,0.81388
loss,0.18455


acc,▁▃▅▇▇███
loss,█▂▁▃▂▁▁▂
acc,0.87368
loss,0.20232


In [16]:
#@title
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset, 
                                         batch_size=batch_size, 
                                         shuffle=True if is_train else False, 
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [17]:
# Launch 5 experiments, trying different dropout rates
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(
        project="pytorch-intro",
        config={
            "epochs": 10,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": random.uniform(0.01, 0.80),
            })
    
    # Copy your config 
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)
    
    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            example_ct += len(images)
            metrics = {"train/train_loss": train_loss, 
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch, 
                       "train/example_ct": example_ct}
            
            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb 
                wandb.log(metrics)
                
            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss, 
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})
        
        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run 
    wandb.finish()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▃▅▅▇▇▇██▇
val/val_loss,█▅▃▂▂▂▁▁▂▂
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.03224
val/val_accuracy,0.951
val/val_loss,0.16758


Train Loss: 0.269, Valid Loss: 0.284818, Accuracy: 0.92
Train Loss: 0.272, Valid Loss: 0.229725, Accuracy: 0.94
Train Loss: 0.234, Valid Loss: 0.209219, Accuracy: 0.94
Train Loss: 0.101, Valid Loss: 0.186123, Accuracy: 0.94
Train Loss: 0.110, Valid Loss: 0.174419, Accuracy: 0.95
Train Loss: 0.084, Valid Loss: 0.175729, Accuracy: 0.95
Train Loss: 0.160, Valid Loss: 0.164836, Accuracy: 0.95
Train Loss: 0.127, Valid Loss: 0.161984, Accuracy: 0.95
Train Loss: 0.051, Valid Loss: 0.161802, Accuracy: 0.95
Train Loss: 0.067, Valid Loss: 0.155771, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▃▄▃▂▂▃▂▂▂▂▂▂▁▂▁▂▁▂▁▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▅▄▆▆▇▇▇██
val/val_loss,█▅▄▃▂▂▁▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.06673
val/val_accuracy,0.9525
val/val_loss,0.15577


Train Loss: 0.405, Valid Loss: 0.292020, Accuracy: 0.92
Train Loss: 0.338, Valid Loss: 0.238453, Accuracy: 0.93
Train Loss: 0.166, Valid Loss: 0.211512, Accuracy: 0.93
Train Loss: 0.232, Valid Loss: 0.203727, Accuracy: 0.94
Train Loss: 0.163, Valid Loss: 0.175842, Accuracy: 0.94
Train Loss: 0.077, Valid Loss: 0.167419, Accuracy: 0.94
Train Loss: 0.150, Valid Loss: 0.157568, Accuracy: 0.95
Train Loss: 0.176, Valid Loss: 0.165095, Accuracy: 0.95
Train Loss: 0.040, Valid Loss: 0.154272, Accuracy: 0.95
Train Loss: 0.098, Valid Loss: 0.153592, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▆▄▃▃▂▃▃▂▂▂▂▂▃▃▂▂▂▁▂▁▂▂▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁
val/val_accuracy,▁▄▄▅▇▇█▇▇█
val/val_loss,█▅▄▄▂▂▁▂▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.0979
val/val_accuracy,0.9505
val/val_loss,0.15359


In [18]:
# Start a wandb run
wandb.init(project="pytorch-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f'Accuracy is: {accuracy}, {acc_threshold}')
    
    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title='Low Accuracy',
            text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',
        )
        print('Alert triggered')
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 0.916, 0.3
Accuracy is: 0.612, 0.3
Accuracy is: 1.336, 0.3
Accuracy is: 1.183, 0.3
Accuracy is: 1.083, 0.3
Accuracy is: 0.32, 0.3
Accuracy is: 1.075, 0.3
Accuracy is: 0.505, 0.3
Accuracy is: 0.603, 0.3
Accuracy is: 1.259, 0.3
Accuracy is: 1.216, 0.3
Accuracy is: 1.106, 0.3
Accuracy is: 0.725, 0.3
Accuracy is: 0.811, 0.3
Accuracy is: 1.269, 0.3
Accuracy is: 0.869, 0.3
Accuracy is: 0.606, 0.3
Accuracy is: 1.537, 0.3
Accuracy is: 1.029, 0.3
Accuracy is: 0.72, 0.3
Accuracy is: 1.599, 0.3
Accuracy is: 1.053, 0.3
Accuracy is: 0.315, 0.3
Accuracy is: 0.442, 0.3
Accuracy is: 1.262, 0.3
Accuracy is: 0.544, 0.3
Accuracy is: 1.372, 0.3
Accuracy is: 0.287, 0.3
Alert triggered


Accuracy,▄▃▇▆▅▁▅▂▃▆▆▅▃▄▆▄▃█▅▃█▅▁▂▆▂▇▁
Accuracy,0.287
